In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import holoviews as hv
import panel as pn
import datetime as dt
from get_data import get_urls_mod, get_urls_obs, get_data

In [ ]:
pn.extension()
hv.extension("bokeh")

# Use-case: Accumulated surface energy fluxes

## Introduction

This notebook contains the code for generating a line-plot for the accumulated surface energy fluxes.

The code responsible for fetching data can be found in get_data.py

This plotting tool has been generated for the Year Of Polar Prediction site Model Intercomparison Project, YOPPsiteMIP

## Code

This following cell contains the code responsible for generating the widgets userd for user interactivity

In [ ]:
#selection of site
site_name = pn.widgets.Select(
    name="Site", options=['sodankyla','tiksi'],value = 'tiksi', margin=(0, 20, 0, 0)
)

#selection of model
model_name = pn.widgets.Select(
    name="model", options=['AROME-Arctic','ifs-ecmwf'],value='ifs-ecmwf', margin=(0, 20, 0, 0)
)

#user selection variable
variable = pn.widgets.Select(
    name="Variables", options=['tas','snd'], margin=(0, 20, 0, 0)
)

#user selection start and end date only sop 1 as of right now
dates = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2018, 2, 1), end=dt.datetime(2018, 3, 31),
    value=(dt.datetime(2018, 2, 1), dt.datetime(2018, 2, 2))
)

#user selection start time for model 
start_time = pn.widgets.Select(
    name="Start time", options=['00','12'], margin=(0, 20, 0, 0)
)

#user selection of which day should be fetched from the model files
#for concat
concat_day = pn.widgets.Select(
    name="Model day selected", options={1:0,2:1,3:2}, margin=(0,20,0,0)
)


loading = pn.indicators.LoadingSpinner(value=False, width=100, height=100)

This following cell contains the code repsonsible for preparing the data, and calculating the accumulated surface energy fluxes

In [ ]:
def data_prep(df):
    
    df_out = pd.DataFrame()
    
    df['hfls'] = df['hfls']*(-1)
    df['hfss'] = df['hfss']*(-1)
    df['rlus'] = df['rlus']*(-1)
    df['rsus'] = df['rsus']*(-1)
    
    calc_acc = df
    
    df_out['netLW'] = df['rlus']+df['rlds']
    df_out['netSW'] = df['rsus']+df['rsds']
    df_out['hfls'] = df['hfls']
    df_out['hfss'] = df['hfss']
    
    df_out = df_out.cumsum()
    
    
    calc_acc = calc_acc.sum(axis=1)
    calc_acc = calc_acc.cumsum()
    df_out['accumulated'] = calc_acc
    
    #number_adds = pd.Series(np.arange(1, len(df_out)+1), df_out.index)
    #df_out = df_out.divide(number_adds, axis = 0)
    
    
    return df_out


This following cell contains the function responsible for generating the line plot.

In [ ]:
def line_plot(data):
    
    plots = []
    for column in data.columns:
        sub = hv.Curve((data.index,data[column]),label=column)
        plots.append(sub)

    plot = hv.Overlay(plots)
    zero = hv.HLine(0)
    zero.opts(
        color = 'black',
        line_width=1.0
    )
    
    plot = plot * zero
    return plot

This following cell contains the main function.

In [ ]:
@pn.depends(dates,start_time,concat_day,model_name,site_name)
def get_plot(dates,start_time,concat_day,model_name,site_name):
    loading.value = True

    variables = ["hfls","hfss","rlus","rlds","rsus","rsds"]

    urls_mod = get_urls_mod(dates[0],dates[1],start_time,model_name,site_name,variables,concat_day)
    data_mod, error_mod = get_data("concatenated",urls_mod,"model",concat_day)

    if error_mod is not None:
        pane = pn.pane.Alert(error_mod)
        loading.value = False
        return pane
    

    st = data_mod['time'][0].values - (data_mod['time'][1].values-data_mod['time'][0].values)
    time_index = data_mod['time'].values
    time_index = np.insert(time_index, 0, st, axis=0)
    
    df = data_mod.to_dataframe()
    df = df.reindex(time_index, fill_value=0)
    
    data = data_prep(df)
    
    plot = line_plot(data)
    
    plot.opts(title="This is a plot",
        ylabel="These are units",
        xlabel="This is time",
        width=600,
        height=400,
        legend_position='right',
        legend_offset=(25,0)
        ) 
    
    loading.value = False
    return plot



This following cell contains the function responsble for deploying the application. 

In [ ]:
pn.serve(pn.Column("insert name here",get_plot, 
                   pn.Row(loading,pn.Column(model_name,site_name,variable,start_time),
                    pn.Column(dates,concat_day)), 
                    width_policy="max"))